In [ ]:
#ライブラリのインポート欄
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import warnings
import seaborn as sns #可視化
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [ ]:
import zipfile#サンプルがzipなので展開する
zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip').extractall() 
zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip').extractall()
#zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip').extractall()

In [ ]:
# CSVを読み込む

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
# 提出用に（分ける）
# データの統合（分けて適用するのが面倒なため）

In [ ]:
#このプログラムは上から実行するので、再実行は上のプログラムからやり直さないとダメ
#予想を表から除いてダミー化
train2 = pd.get_dummies(train['type'])
del train['type']
COLOR = pd.get_dummies(train['color'])
del train['color']
del train['id']
#ダミー化した結果をクラスとして格納
target = pd.DataFrame(train2['Ghost']* 0 + train2['Ghoul'] * 2 + train2['Goblin'] * 1)
target_GOB = pd.DataFrame(train2['Ghost']* 0 + train2['Ghoul'] * 0 + train2['Goblin'] * 1)
target = target.rename(columns={'0': 'target'})
#予想を表から除いてダミー化(テストデータも)
COLOR2 = pd.get_dummies(test['color'])
del test['color']
ID = test["id"]
del test['id']

In [ ]:
import numpy as np
%matplotlib inline
from sklearn import datasets
from sklearn.model_selection import train_test_split # クロスバリデーション用（テストとトレ分ける）
from sklearn.model_selection import cross_val_score
from sklearn import metrics       # 精度検証用
from sklearn.metrics import accuracy_score

# ニューラルネットワーク
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import joblib

In [ ]:
#ゴブリンだけ色を入れて予想
train_c = train.join(COLOR)
test_c = test.join(COLOR2)
#target_c = target.join(COLOR)

In [ ]:
def GOB():
    global y_pred_c
    global train
    global test
    # paramate
    hidden_layer_sizes=(100,)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True

    # 説明変数

    train_X = train_c
    # 目的変数

    train_y1 = target_GOB

    # 学習
    clf = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )

    clf.fit(train_X, train_y1)

    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    # 学習結果を出力
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)

    # 学習済ファイルのロード
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)

    # 検証用の説明変数

    test_X1 = test_c
    from sklearn import preprocessing
    # 学習結果の検証
    #predict_y1 = clf1.predict_proba(train_X)
    predict_Y1 = clf1.predict(train_X)
    print(predict_Y1)
    
    y_pred_c = pd.DataFrame(predict_Y1)
    train = train.join(y_pred_c)
    
    #predict_y1 = clf1.predict_proba(test_X1)
    predict_Y1 = clf1.predict(test_X1)
    print(predict_Y1)
    y_pred_c = pd.DataFrame(predict_Y1)
    test = test.join(y_pred_c)

In [ ]:
dataset = pd.concat([train, test], ignore_index = True)
#print('isnull\n',train.isnull().sum())#欠損値がないことを確認

In [ ]:
 def lightGBM_k():
    global y_pred_AA
    # paramate
    hidden_layer_sizes=(100,)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True

    # 説明変数

    train_X = train_k
    # 目的変数

    train_y1 = target_k

    # 学習
    clf = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )

    clf.fit(train_X, train_y1)

    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    # 学習結果を出力
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)

    # 学習済ファイルのロード
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)

    # 検証用の説明変数

    test_X1 = test_k

    # 学習結果の検証
   # predict_y1 = clf1.predict_proba(test_X1)
    predict_Y = clf1.predict(test_X1)
    print(predict_Y)
     
    predict_Y = np.round(predict_Y, decimals=1) # 最尤と判断したクラスの値にする
    predict_Y1 = np.round(predict_Y)
    print(predict_Y1)
    y_pred_AA = predict_Y1
    # 学習データの精度
    print("学習データの精度: {:.3f}".format(clf.score(train_X, train_y1)))

    # テスト結果の精度
    print("テスト結果の精度: {:.3f}".format(clf.score(test_X1, predict_Y1)))

In [ ]:
def syuunou():
    global s_c
    global y_pred_AA
    y_pred_z:str=[]
    for n in range(len(y_pred_AA)):
        if y_pred_AA[n]==0:
            y_pred_z.append('Ghost')
        if y_pred_AA[n]==1:
            y_pred_z.append('Ghoul')
        if y_pred_AA[n]==2:
            y_pred_z.append('Goblin')
            
    
    
    s_c= pd.DataFrame({"id": ID_black, "type": y_pred_z})

In [ ]:
def color_select():
    global train
    global test
    global target
    global train_k
    global test_k
    global target_k
    global lightGBM_k
    global syuunou
    global ID_black
    global y_pred_AA
    #train_k = train[train[color] == 1]
    #test_k = test[test[color] == 1]
    #target_k = target[target[color] == 1]
    train_k = train
    test_k = test
    target_k = target
    ID_black = ID
    lightGBM_k()
    syuunou()

In [ ]:
#各色で選択する
#GOB()
color_select()
sc = s_c


#並べ替え　初期設定
rows = sc.sort_values('id')




In [ ]:
##ここから推理します


# Kaggle提出用csvファイルの作成







#submission = pd.DataFrame({"id": ID, "type": y_preda})
submission = rows
submission.to_csv("submission6.csv", index=False)